In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:
# Load and preprocess CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0


170498071/170498071 [==============================] - 2s 0us/step


In [3]:
# Define Inception module as a separate model
def InceptionModule(x, filters):
    branch1x1 = layers.Conv2D(filters[0], (1, 1), padding='same', activation='relu')(x)

    branch3x3 = layers.Conv2D(filters[1], (1, 1), padding='same', activation='relu')(x)
    branch3x3 = layers.Conv2D(filters[2], (3, 3), padding='same', activation='relu')(branch3x3)

    branch5x5 = layers.Conv2D(filters[3], (1, 1), padding='same', activation='relu')(x)
    branch5x5 = layers.Conv2D(filters[4], (5, 5), padding='same', activation='relu')(branch5x5)

    branch_pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    branch_pool = layers.Conv2D(filters[5], (1, 1), padding='same', activation='relu')(branch_pool)

    return layers.concatenate([branch1x1, branch3x3, branch5x5, branch_pool], axis=-1)


In [4]:
# Build the model using the functional API
input_layer = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, (3, 3), activation='relu')(input_layer)
x = InceptionModule(x, [64, 128, 128, 32, 32, 32])
x = layers.GlobalAveragePooling2D()(x)
output_layer = layers.Dense(10, activation='softmax')(x)

model = models.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [5]:
# Train the model
model.fit(train_images, train_labels, epochs=20, validation_data=(test_images, test_labels))

Epoch 1/20
1563/1563 [==============================] - 30s 15ms/step - loss: 1.8125 - accuracy: 0.3138 - val_loss: 1.6539 - val_accuracy: 0.3919
Epoch 2/20
1563/1563 [==============================] - 22s 14ms/step - loss: 1.5876 - accuracy: 0.4213 - val_loss: 1.5385 - val_accuracy: 0.4322
Epoch 3/20
1563/1563 [==============================] - 23s 15ms/step - loss: 1.4634 - accuracy: 0.4752 - val_loss: 1.4663 - val_accuracy: 0.4820
Epoch 4/20
1563/1563 [==============================] - 23s 15ms/step - loss: 1.3860 - accuracy: 0.5050 - val_loss: 1.4498 - val_accuracy: 0.4802
Epoch 5/20
1563/1563 [==============================] - 22s 14ms/step - loss: 1.3341 - accuracy: 0.5287 - val_loss: 1.3376 - val_accuracy: 0.5238
Epoch 6/20
1563/1563 [==============================] - 22s 14ms/step - loss: 1.2884 - accuracy: 0.5452 - val_loss: 1.2630 - val_accuracy: 0.5520
Epoch 7/20
1563/1563 [==============================] - 23s 14ms/step - loss: 1.2470 - accuracy: 0.5613 - val_loss: 1.2810 -

In [6]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

313/313 [==============================] - 2s 5ms/step - loss: 0.9912 - accuracy: 0.6543
Test Accuracy: 65.43%
